In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ChromeService as Service
from bs4 import BeautifulSoup
import re
import time
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

def initialize_chromedriver() -> webdriver.Chrome:
    prefs = {"profile.default_content_setting_values.geolocation": 2}
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--deny-permission-prompts")
    options.add_experimental_option("prefs", prefs)
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
    except:
        raise Exception("Failed to install ChromeDriver")
    return driver

In [103]:
driver = initialize_chromedriver()

Exception: Failed to install ChromeDriver

In [74]:
driver.get("https://www.booking.com/searchresults.html?ss=Ho+Chi+Minh+City%2C+Ho+Chi+Minh+Municipality%2C+Vietnam&ssne=Hue&ssne_untouched=Hue&label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKc2IunBsACAdICJDYyMmU4MGQ1LWUzYWQtNGRhZC1iNmEzLWJhOTI5OWVjZTQ1YtgCBeACAQ&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-3730078&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=a98d21b625600091&ac_meta=GhBhOThkMjFiNjI1NjAwMDkxIAAoATICZW46C0hvIENoaSBNaW5oQABKAFAA&checkin=2024-11-21&checkout=2024-11-22&group_adults=2&no_rooms=1&group_children=0")

In [75]:
# extract hotel links
hotel_links = []
for i in range(1, 5):
	driver.execute_script(f"window.scrollTo(0, {i*1000})")
	time.sleep(1)
	soup = BeautifulSoup(driver.page_source, 'html.parser')
	for a in soup.find_all('a', href=True):
		if re.search(r'/hotel/', a['href']):
			hotel_links.append(a['href'])
            
hotel_links = list(set(hotel_links))
hotel_links

['https://www.booking.com/hotel/vn/arium-saigon.html',
 'https://www.booking.com/hotel/vn/sotetsu-grand-fresa-saigon.html',
 'https://www.booking.com/hotel/vn/la-jolie.html',
 'https://www.booking.com/hotel/vn/saigon-apartment-quan-1.html',
 'https://www.booking.com/hotel/vn/la-vela-saigon.html',
 'https://www.booking.com/hotel/vn/the-white-guesthouse-ho-chi-minh.html',
 'https://www.booking.com/hotel/vn/hoang-lan-ho-chi-minh-city.html',
 'https://www.booking.com/hotel/vn/ha-noi-green.html',
 'https://www.booking.com/hotel/vn/orchids.html',
 'https://www.booking.com/hotel/vn/mia-quan-11.html',
 'https://www.booking.com/hotel/vn/silverland-benthanh.html',
 'https://www.booking.com/hotel/vn/sedona-suites-grand-tower.html',
 'https://www.booking.com/hotel/vn/the-river-park.html',
 'https://www.booking.com/hotel/vn/paradise-saigon-boutique.html',
 'https://www.booking.com/hotel/vn/hanz-upstair-mpchill.html',
 'https://www.booking.com/hotel/vn/unique-star-boutique.html',
 'https://www.booki

In [79]:
# test one hotel for further function
target_url = hotel_links[0]

# using driver to get the page source
driver.get(target_url)
WebDriverWait(driver, 10).until(lambda d: d.execute_script('return document.readyState') == 'complete')

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [84]:
soup.find('table', {'class':'hprt-container'})

AttributeError: 'NoneType' object has no attribute 'count'

In [66]:

o = {}
o['link'] = str(target_url)
try:
    o["name"] = soup.find("h2", {"class": "pp-header__title"}).text
except:
    o["name"] = np.nan
try:
    o["address"] = soup.find("span", {"class": "hp_address_subtitle"}).text.strip("\n")
except:
    o["address"] = np.nan
try:
    o["content"] = soup.find("div", {"class": "hp_desc_main_content"}).text.strip()
except:
    o["content"] = np.nan
try:
    rs, rc = soup.find('div', {'data-testid': 'review-score-component'})
    o['review_score'] = rs.text.strip()
except:
    o['review_score'] = np.nan
try:
    o['review_count'] = rc.text.split("\xa0·\xa0")[-1]
except:
    o['review_count'] = np.nan

try:
    fac = soup.find("div", {"data-testid": "property-most-popular-facilities-wrapper"})
    fac_arr = []
    for i in range(len(fac.find_all("span"))):
        all_fac = fac.find_all("span")
        text = all_fac[i].text.strip()
        if text != "" and text not in fac_arr:
            fac_arr.append(text)
    o['most_facility'] = str(fac_arr)
except:
    o['most_facility'] = np.nan

try:
    all_facilities = []
    for each in soup.find_all("div", {"class": "e50d7535fa"}):
        t = each.find('div', {'class': 'd1ca9115fe'}).text
        rs = []
        for a in each.find_all("span"):
            x = a.text.replace(t, '').strip()
            if x != '' and x not in rs:
                rs.append(a.text.replace(t, '').strip())
        all_facilities.append({'key': t, 'value': rs})
    o['all_facilities'] = all_facilities
except:
    o['all_facilities'] = np.nan

o

['{"chunks":["b9a82cb8","81da5f32","dc32f6b7","be127943","6b97af9b","8d7d1278","09fb2abd","21366851","7e8161ba","d7b33d94","6e375f45","27edabe6","0cb2af2c","1840e569","1f16ef0c","28b9444a","045b19e3","ed63799f","9e5951df","c009a34e","11334689","f5af45d7","36d4789c","5de579b9","2b8df162","fbca8bad","f1f5425f","820c8d43","1055d887","5d8d6d86","51190c07","db46a508","037d4f1a","fdf2a8d6","16d0c18c","00e71363","d703c930","232069f7","5c91240d","111310cc","6e97451b","b7d32ec3","1fdcbf2b","195efcdb","f7bdfc98","bea60ab5","5075d9e8","fbebfdab","21f1fc11","8a4e947b","ac8c03a1","2cea9793","9248d82a","538e3f03","a5f265fc","00f0620c","64eba63e","2754406a","04da5492","532cdb87","950de38a","f0477014","44d0a763","e1a75ad8","43032106","42ba0fc1","db6ecff5","47a97127","d11285c8","7a58925b","8094fb6a","9e196822","c7560c9f","35bc4f10","00dc3ab8","ee9136df","9ba6ecd9","ba5ad9a8","fd4863eb","caecd84c","cda06c20","9a13b4cf","34e01dac","e2b56b8b","6d60dfbd","801ef4dc","7d81a97d","88c49b18","2b5550ad","c941794